# Variable Preprocessing

In [1]:
# install required libraries
# !pip install xarray netCDF4 h5netcdf numpy rioxarray

In [2]:
import xarray as xr
import numpy as np

### Load ERA5-Land and ERA5 data

In [3]:
era5land = xr.open_dataset('era5-land.nc') # open ERA5-Land NetCDF file
print(era5land) # view structure
print("\n--------------------------------------------------------------------------------\n")

era5 = xr.open_dataset("era5.nc") # open ERA5 NetCEF file
print(era5) # view structure

<xarray.Dataset> Size: 180MB
Dimensions:     (valid_time: 744, latitude: 96, longitude: 105)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 2019-08-01 ... 2019-08-31T23:...
  * latitude    (latitude) float64 768B 42.0 41.9 41.8 41.7 ... 32.7 32.6 32.5
  * longitude   (longitude) float64 840B -124.5 -124.4 -124.3 ... -114.2 -114.1
    number      int64 8B ...
    expver      (valid_time) <U4 12kB ...
Data variables:
    d2m         (valid_time, latitude, longitude) float32 30MB ...
    t2m         (valid_time, latitude, longitude) float32 30MB ...
    ssrd        (valid_time, latitude, longitude) float32 30MB ...
    u10         (valid_time, latitude, longitude) float32 30MB ...
    v10         (valid_time, latitude, longitude) float32 30MB ...
    sp          (valid_time, latitude, longitude) float32 30MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conv

### convert varaible units

In [4]:
# convert t2m from K to Celsius -- used as "Tair" input
t2m_c = era5land["t2m"] - 273.15
t2m_c.attrs["units"] = "C"

# convert d2m from K to Celsius -- used in "relhum" calculation
d2m_c = era5land["d2m"] - 273.15
d2m_c.attrs["units"] = "C"

# convert ssrd from J/m2 to W/m2 -- used as "solar" input
era5land["ssrd"].valid_time.diff("time") # check that time resolution is hourly
ssrd_W = era5land["ssrd"] / 3600         # dividy by 3600 seconds since hourly accumulated
ssrd_W.attrs["units"] = "W m**-2"

# convert sp from Pa to hPa -- used as "pres" input
sp_hpa = era5land["sp"] / 100
sp_hpa.attrs["units"] = "hPa"

### calculate derived variables

In [5]:
# calculate relative humidity (%) -- used as "relhum" input
vapor_pres = 610.94*np.exp(17.625*d2m_c / (243.04+d2m_c))
sat_vapor_pres = 610.94*np.exp(17.625*t2m_c / (243.04+t2m_c))
rh = 100*(vapor_pres/sat_vapor_pres)
rh.attrs["long_name"] = "relative humidity (%)"
rh.attrs["units"] = "%"

# calculate wind speed (m/s) -- used as "speed" input
u = era5land["u10"]
v = era5land["v10"]
ws = np.sqrt(u**2 + v**2)
ws.attrs["long_name"] = "wind speed (m/s)"
ws.attrs["units"] = "m s**-1"

In [6]:
# load interpolated ERA5 fdir nc file
era5_fdir_idw = xr.open_dataset('era5_fdir_idw_2019.nc') 
print(era5_fdir_idw)

<xarray.Dataset> Size: 30MB
Dimensions:    (time: 744, latitude: 96, longitude: 105)
Coordinates:
  * time       (time) datetime64[ns] 6kB 2019-08-01 ... 2019-08-31T23:00:00
  * latitude   (latitude) float64 768B 42.0 41.9 41.8 41.7 ... 32.7 32.6 32.5
  * longitude  (longitude) float64 840B -124.5 -124.4 -124.3 ... -114.2 -114.1
Data variables:
    fdir       (time, latitude, longitude) float32 30MB ...
Attributes:
    source:         ERA5
    target_grid:    era5land_d2m_2019-08-01T00.tif
    interpolation:  IDW (k=8, power=2.0) in lon/lat degree space
    crs:            EPSG:4326


In [19]:
# calculate fraction of surface solar radiation that is direct (0-1) -- used as "fdir" input

#print(era5_fdir_idw["fdir"].attrs.get("units"))
#print(era5land["ssrd"].attrs.get("units"))       # check that numerator and denomator are of the same units

# fraction calculated as fdir/ssrd, nan if ssrd is nan, 0 if ssrd is 0
fdir = xr.where(era5land["ssrd"].isnull(), np.nan, 
                xr.where(era5land["ssrd"] > 0, era5_fdir_idw["fdir"].values / era5land["ssrd"].values, 0.0))
fdir.attrs["long_name"] = "fraction of surface solar radiation that is direct (0-1)"
fdir.attrs["units"] = ""  # no units

### obtain and derive "urban" variable

In [8]:
# from National Land Cover Database, https://www.usgs.gov/centers/eros/science/national-land-cover-database

### build a working dataset containing WBGT input variables
xr.Dataset creates a dataset resembling an in-memory representation of a NetCDF file. Consists of variables, coordinates, and attributes, which together form a self-describing dataset.

In [9]:
# check resolution of variables before putting into working dataset

print(abs(ssrd_W.latitude[1] - ssrd_W.latitude[0]).values)
print(abs(fdir.latitude[1] - fdir.latitude[0]).values)
print(abs(sp_hpa.latitude[1] - sp_hpa.latitude[0]).values)
print(abs(t2m_c.latitude[1] - t2m_c.latitude[0]).values)
print(abs(rh.latitude[1] - rh.latitude[0]).values)
print(abs(ws.latitude[1] - ws.latitude[0]).values)
# urban

0.10000000000000142
0.10000000000000142
0.10000000000000142
0.10000000000000142
0.10000000000000142
0.10000000000000142


In [10]:
# zspeed = 10 m and dt = −0.052 C don't need to be in ds_work since they are fixed parameters (don't change through space and time)
# They can be inputted into the WBGT function as constants.

# cza input will be calculated using the calc_cza_int(lat, lon, y, mon, d, hr) function in the heatmetrics R package

ds_work = xr.Dataset(     # NEED TO ADD: urban
    {"solar": ssrd_W,
     "fdir": fdir,
     "pres": sp_hpa,
     "Tair": t2m_c,
     "relhum": rh,
     "speed": ws
     # "urban": ...
    }
    )

# Create needed coordinates
time = ds_work.valid_time
ds_work = ds_work.assign_coords(
    year = time.dt.year,
    month = time.dt.month,
    day=time.dt.day, # day of month (whole number) and hour used as input to the calc_cza_int function in the heatmetrics R package
    hour=time.dt.hour,
    # decimal day of month (UTC)
    dday = (time.dt.day
            + time.dt.hour / 24
            + time.dt.minute / 1440
            + time.dt.second / 86400))

print(ds_work)  # view structure

<xarray.Dataset> Size: 180MB
Dimensions:     (valid_time: 744, latitude: 96, longitude: 105)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 2019-08-01 ... 2019-08-31T23:...
  * latitude    (latitude) float64 768B 42.0 41.9 41.8 41.7 ... 32.7 32.6 32.5
  * longitude   (longitude) float64 840B -124.5 -124.4 -124.3 ... -114.2 -114.1
    number      int64 8B 0
    expver      (valid_time) <U4 12kB '0001' '0001' '0001' ... '0001' '0001'
    year        (valid_time) int64 6kB 2019 2019 2019 2019 ... 2019 2019 2019
    month       (valid_time) int64 6kB 8 8 8 8 8 8 8 8 8 8 ... 8 8 8 8 8 8 8 8 8
    day         (valid_time) int64 6kB 1 1 1 1 1 1 1 1 ... 31 31 31 31 31 31 31
    hour        (valid_time) int64 6kB 0 1 2 3 4 5 6 7 ... 17 18 19 20 21 22 23
    dday        (valid_time) float64 6kB 1.0 1.042 1.083 ... 31.88 31.92 31.96
Data variables:
    solar       (valid_time, latitude, longitude) float32 30MB nan ... 6.541e+03
    fdir        (valid_time, latitude, longitude) float

In [11]:
# check long name and units of preprocessed variables
print("solar", ds_work["solar"].attrs["long_name"], ds_work["solar"].attrs["units"])
print("fdir", ds_work["fdir"].attrs["long_name"], ds_work["fdir"].attrs["units"])
print("pres", ds_work["pres"].attrs["long_name"], ds_work["pres"].attrs["units"])
print("Tair", ds_work["Tair"].attrs["long_name"], ds_work["Tair"].attrs["units"])
print("relhum", ds_work["relhum"].attrs["long_name"], ds_work["relhum"].attrs["units"])
print("speed", ds_work["speed"].attrs["long_name"], ds_work["speed"].attrs["units"])
# urban

solar Surface short-wave (solar) radiation downwards W m**-2
fdir fraction of surface solar radiation that is direct (0-1) 
pres Surface pressure hPa
Tair 2 metre temperature C
relhum relative humidity (%) %
speed wind speed (m/s) m s**-1


### view values of preprocessed variables

In [12]:
print(ds_work["solar"])

<xarray.DataArray 'solar' (valid_time: 744, latitude: 96, longitude: 105)> Size: 30MB
array([[[       nan,        nan,        nan, ..., 6873.277  ,
         6807.43   , 6742.1187 ],
        [       nan,        nan,        nan, ..., 6875.0034 ,
         6817.309  , 6759.7866 ],
        [       nan,        nan,        nan, ..., 6876.4106 ,
         6827.0884 , 6778.125  ],
        ...,
        [       nan,        nan,        nan, ..., 6219.4985 ,
         6157.0254 , 6130.173  ],
        [       nan,        nan,        nan, ..., 6255.257  ,
         6177.722  , 6142.894  ],
        [       nan,        nan,        nan, ..., 6288.5493 ,
         6204.0557 , 6156.1846 ]],

       [[       nan,        nan,        nan, ...,  260.15952,
          249.36958,  238.71764],
        [       nan,        nan,        nan, ...,  252.9216 ,
          242.4748 ,  232.44395],
        [       nan,        nan,        nan, ...,  245.67021,
          235.61008,  225.9593 ],
...
        [       nan,        nan

In [13]:
print(ds_work["fdir"])

<xarray.DataArray 'fdir' (valid_time: 744, latitude: 96, longitude: 105)> Size: 30MB
array([[[       nan,        nan,        nan, ..., 0.05086216,
         0.05122638, 0.05075682],
        [       nan,        nan,        nan, ..., 0.04985529,
         0.04990999, 0.04900372],
        [       nan,        nan,        nan, ..., 0.04785676,
         0.04759393, 0.0473007 ],
        ...,
        [       nan,        nan,        nan, ..., 0.02374677,
         0.02305963, 0.02603178],
        [       nan,        nan,        nan, ..., 0.02697981,
         0.02623949, 0.02690857],
        [       nan,        nan,        nan, ..., 0.02581596,
         0.025716  , 0.02742993]],

       [[       nan,        nan,        nan, ..., 0.36982337,
         0.38157013, 0.39081016],
        [       nan,        nan,        nan, ..., 0.36474496,
         0.3691162 , 0.37126225],
        [       nan,        nan,        nan, ..., 0.3098174 ,
         0.30894798, 0.346548  ],
...
        [       nan,        nan,

In [14]:
print(ds_work["pres"])

<xarray.DataArray 'pres' (valid_time: 744, latitude: 96, longitude: 105)> Size: 30MB
array([[[      nan,       nan,       nan, ..., 810.78687, 819.1269 ,
         828.6769 ],
        [      nan,       nan,       nan, ..., 815.4269 , 827.02686,
         831.45685],
        [      nan,       nan,       nan, ..., 813.9469 , 817.89685,
         819.97687],
        ...,
        [      nan,       nan,       nan, ..., 992.52686, 995.20685,
         999.6869 ],
        [      nan,       nan,       nan, ..., 985.51685, 986.08685,
         991.2469 ],
        [      nan,       nan,       nan, ..., 981.6869 , 978.7369 ,
         981.76685]],

       [[      nan,       nan,       nan, ..., 810.6725 , 819.2125 ,
         829.0025 ],
        [      nan,       nan,       nan, ..., 815.4425 , 827.3325 ,
         831.9525 ],
        [      nan,       nan,       nan, ..., 814.0425 , 818.2325 ,
         820.4725 ],
...
        [      nan,       nan,       nan, ..., 989.6188 , 992.10876,
         996.4387

In [15]:
print(ds_work["Tair"])

<xarray.DataArray 'Tair' (valid_time: 744, latitude: 96, longitude: 105)> Size: 30MB
array([[[      nan,       nan,       nan, ..., 31.408356, 32.062653,
         32.94742 ],
        [      nan,       nan,       nan, ..., 31.963043, 32.92398 ,
         33.04117 ],
        [      nan,       nan,       nan, ..., 31.996246, 32.127106,
         31.92984 ],
        ...,
        [      nan,       nan,       nan, ..., 35.164215, 34.87906 ,
         34.982574],
        [      nan,       nan,       nan, ..., 34.92398 , 34.459137,
         34.453278],
        [      nan,       nan,       nan, ..., 34.838043, 34.140778,
         33.902496]],

       [[      nan,       nan,       nan, ..., 30.099518, 29.933502,
         30.130768],
        [      nan,       nan,       nan, ..., 29.947174, 29.865143,
         29.312408],
        [      nan,       nan,       nan, ..., 29.224518, 28.39444 ,
         27.537018],
...
        [      nan,       nan,       nan, ..., 41.93106 , 42.08731 ,
         42.52285

In [16]:
print(ds_work["relhum"])

<xarray.DataArray 'relhum' (valid_time: 744, latitude: 96, longitude: 105)> Size: 30MB
array([[[       nan,        nan,        nan, ...,  9.197327 ,
          9.594236 ,  9.426522 ],
        [       nan,        nan,        nan, ...,  8.724601 ,
          9.165896 ,  9.959862 ],
        [       nan,        nan,        nan, ...,  8.511751 ,
          9.204394 , 10.24743  ],
        ...,
        [       nan,        nan,        nan, ..., 38.450275 ,
         40.254673 , 40.873302 ],
        [       nan,        nan,        nan, ..., 40.144775 ,
         42.6273   , 43.63656  ],
        [       nan,        nan,        nan, ..., 41.774784 ,
         44.649635 , 46.3833   ]],

       [[       nan,        nan,        nan, ..., 11.216    ,
         13.518557 , 15.518783 ],
        [       nan,        nan,        nan, ..., 13.276413 ,
         16.341812 , 19.347889 ],
        [       nan,        nan,        nan, ..., 15.584226 ,
         19.740896 , 23.758755 ],
...
        [       nan,        na

In [17]:
print(ds_work["speed"])

<xarray.DataArray 'speed' (valid_time: 744, latitude: 96, longitude: 105)> Size: 30MB
array([[[      nan,       nan,       nan, ..., 2.2982075, 2.7077718,
         3.1220136],
        [      nan,       nan,       nan, ..., 3.3003225, 3.9200892,
         4.427648 ],
        [      nan,       nan,       nan, ..., 4.4253554, 5.2548637,
         5.9585514],
        ...,
        [      nan,       nan,       nan, ..., 3.9846215, 3.280328 ,
         2.1595635],
        [      nan,       nan,       nan, ..., 3.510377 , 2.990506 ,
         1.9416866],
        [      nan,       nan,       nan, ..., 3.17321  , 2.7572112,
         1.8545384]],

       [[      nan,       nan,       nan, ..., 4.2914753, 5.258579 ,
         6.08136  ],
        [      nan,       nan,       nan, ..., 5.688283 , 6.646674 ,
         7.1297708],
        [      nan,       nan,       nan, ..., 7.0884666, 7.9883633,
         8.485993 ],
...
        [      nan,       nan,       nan, ..., 3.0654716, 2.9220061,
         2.71552